# P22 IDS - Model Training and Hyperparameter Tuning

This notebook demonstrates advanced model training features.

## What You'll Learn:
1. Hyperparameter tuning with Optuna
2. K-Fold cross-validation
3. Model training with early stopping
4. Visualize training history
5. Save and load trained models

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from services.modelTrainingService import ModelTrainingService
from orchestrator import ServiceOrchestrator

print("✓ Imports successful!")

## 1. Prepare Training Data

In [ ]:
# Create synthetic training data for demonstration
# Replace with your actual dataset

np.random.seed(42)

# Generate synthetic features (1000 samples, 20 features)
n_samples = 1000
n_features = 20
n_classes = 5

# Normal traffic (class 0)
normal_features = np.random.normal(0.5, 0.1, (600, n_features))
normal_labels = np.zeros(600, dtype=int)

# Attack traffic (classes 1-4)
attack_features = np.random.normal(0.7, 0.15, (400, n_features))
attack_labels = np.random.randint(1, n_classes, 400)

# Combine
features = np.vstack([normal_features, attack_features])
labels = np.hstack([normal_labels, attack_labels])

# Shuffle
indices = np.random.permutation(n_samples)
features = features[indices]
labels = labels[indices]

print(f"✓ Created training data:")
print(f"  Samples: {features.shape[0]}")
print(f"  Features: {features.shape[1]}")
print(f"  Classes: {len(np.unique(labels))}")
print(f"  Class distribution: {dict(zip(*np.unique(labels, return_counts=True)))}")

## 2. Hyperparameter Tuning with Optuna

In [ ]:
# Initialize training service
trainer = ModelTrainingService({
    'numEpochs': 50,
    'batchSize': 32,
    'outputDir': '../outputs/training'
})

trainer.start()
print("✓ Training service initialized")

In [ ]:
# Run hyperparameter tuning for LSTM
print("Starting hyperparameter tuning for LSTM...")
print("This may take several minutes...\n")

lstm_tuning_results = trainer.process({
    'action': 'tune',
    'features': features,
    'labels': labels,
    'modelType': 'lstm',
    'nTrials': 20  # Reduce for faster execution
})

print("\n" + "="*50)
print("LSTM TUNING RESULTS")
print("="*50)
print(f"\nBest Parameters:")
for param, value in lstm_tuning_results['bestParams'].items():
    print(f"  {param}: {value}")
print(f"\nBest Score: {lstm_tuning_results['bestScore']:.4f}")
print(f"Results saved to: {lstm_tuning_results['resultsPath']}")

In [ ]:
# Visualize tuning results
with open(lstm_tuning_results['resultsPath']) as f:
    tuning_data = json.load(f)

# Extract trial scores
trial_numbers = [t['number'] for t in tuning_data['allTrials']]
trial_scores = [t['value'] for t in tuning_data['allTrials']]

# Plot
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(trial_numbers, trial_scores, 'o-', alpha=0.6)
plt.axhline(y=tuning_data['bestScore'], color='r', linestyle='--', 
            label=f"Best: {tuning_data['bestScore']:.4f}")
plt.title('Hyperparameter Tuning Progress')
plt.xlabel('Trial Number')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(trial_scores, bins=15, color='steelblue', edgecolor='black', alpha=0.7)
plt.axvline(x=tuning_data['bestScore'], color='r', linestyle='--', 
            label=f"Best: {tuning_data['bestScore']:.4f}")
plt.title('Score Distribution')
plt.xlabel('Validation Accuracy')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

## 3. K-Fold Cross-Validation

In [ ]:
# Run 5-fold cross-validation with best parameters
print("Running 5-Fold Cross-Validation...\n")

best_params = lstm_tuning_results['bestParams']

cv_results = trainer.process({
    'action': 'cross_validate',
    'features': features,
    'labels': labels,
    'modelType': 'lstm',
    'config': {
        'inputSize': features.shape[1],
        'numClasses': len(np.unique(labels)),
        **{k: v for k, v in best_params.items() if k != 'learningRate'}
    }
})

print("\n" + "="*50)
print("CROSS-VALIDATION RESULTS")
print("="*50)

avg_metrics = cv_results['averageMetrics']
print(f"\nAccuracy:  {avg_metrics['accuracy']:.4f} ± {avg_metrics['std_accuracy']:.4f}")
print(f"Precision: {avg_metrics['precision']:.4f}")
print(f"Recall:    {avg_metrics['recall']:.4f}")
print(f"F1-Score:  {avg_metrics['f1Score']:.4f} ± {avg_metrics['std_f1Score']:.4f}")

print(f"\nResults saved to: {cv_results['resultsPath']}")

In [ ]:
# Visualize cross-validation results
fold_results = cv_results['foldResults']
fold_numbers = [r['fold'] for r in fold_results]
accuracies = [r['accuracy'] for r in fold_results]
f1_scores = [r['f1Score'] for r in fold_results]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy per fold
axes[0].bar(fold_numbers, accuracies, color='lightblue', edgecolor='black')
axes[0].axhline(y=avg_metrics['accuracy'], color='r', linestyle='--', 
                label=f"Avg: {avg_metrics['accuracy']:.4f}")
axes[0].fill_between([0.5, 5.5], 
                      avg_metrics['accuracy'] - avg_metrics['std_accuracy'],
                      avg_metrics['accuracy'] + avg_metrics['std_accuracy'],
                      alpha=0.2, color='red')
axes[0].set_title('Accuracy per Fold')
axes[0].set_xlabel('Fold')
axes[0].set_ylabel('Accuracy')
axes[0].set_ylim([min(accuracies) - 0.05, max(accuracies) + 0.05])
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# F1-Score per fold
axes[1].bar(fold_numbers, f1_scores, color='lightcoral', edgecolor='black')
axes[1].axhline(y=avg_metrics['f1Score'], color='r', linestyle='--',
                label=f"Avg: {avg_metrics['f1Score']:.4f}")
axes[1].fill_between([0.5, 5.5],
                      avg_metrics['f1Score'] - avg_metrics['std_f1Score'],
                      avg_metrics['f1Score'] + avg_metrics['std_f1Score'],
                      alpha=0.2, color='red')
axes[1].set_title('F1-Score per Fold')
axes[1].set_xlabel('Fold')
axes[1].set_ylabel('F1-Score')
axes[1].set_ylim([min(f1_scores) - 0.05, max(f1_scores) + 0.05])
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Train Final Model with Best Parameters

In [ ]:
# Train final model with early stopping
print("Training final model with best parameters...\n")

training_results = trainer.process({
    'action': 'train',
    'features': features,
    'labels': labels,
    'modelType': 'lstm',
    'config': {
        'inputSize': features.shape[1],
        'numClasses': len(np.unique(labels)),
        **{k: v for k, v in best_params.items() if k != 'learningRate'}
    }
})

print("\n" + "="*50)
print("TRAINING COMPLETE")
print("="*50)
print(f"\nBest Validation Accuracy: {training_results['bestValAccuracy']:.4f}")
print(f"Model saved to: {training_results['checkpointPath']}")

final_metrics = training_results['finalMetrics']
print(f"\nFinal Metrics:")
print(f"  Accuracy:  {final_metrics['accuracy']:.4f}")
print(f"  Precision: {final_metrics['precision']:.4f}")
print(f"  Recall:    {final_metrics['recall']:.4f}")
print(f"  F1-Score:  {final_metrics['f1Score']:.4f}")

## 5. Visualize Training History

In [ ]:
# Plot training history
history = training_results['trainingHistory']

epochs = range(1, len(history['train_loss']) + 1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
axes[0].plot(epochs, history['train_loss'], 'b-', label='Training Loss', linewidth=2)
axes[0].plot(epochs, history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Accuracy curves
axes[1].plot(epochs, history['train_acc'], 'b-', label='Training Accuracy', linewidth=2)
axes[1].plot(epochs, history['val_acc'], 'r-', label='Validation Accuracy', linewidth=2)
axes[1].axhline(y=training_results['bestValAccuracy'], color='g', linestyle='--',
                label=f"Best Val Acc: {training_results['bestValAccuracy']:.4f}")
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../training_curves.png', dpi=150)
print("✓ Training curves saved to: training_curves.png")
plt.show()

## 6. Load and Test Trained Model

In [ ]:
# Create test data
test_features = np.random.normal(0.6, 0.12, (100, n_features))
test_labels = np.random.randint(0, n_classes, 100)

print(f"Created test set: {test_features.shape[0]} samples")

In [ ]:
# Load trained model and run inference
from services.lstmModelService import LSTMModelService

lstm_service = LSTMModelService({
    'inputSize': features.shape[1],
    'numClasses': len(np.unique(labels)),
    **{k: v for k, v in best_params.items() if k != 'learningRate'},
    'modelPath': training_results['checkpointPath']
})

lstm_service.start()
print("✓ Loaded trained model")

# Run prediction
test_results = lstm_service.process({
    'features': test_features,
    'metadata': {'source': 'test_set'}
})

print(f"\nTest Predictions (first 20): {test_results['predictions'][:20]}")
print(f"Average Confidence: {np.mean(test_results['confidences']):.4f}")

# Calculate test accuracy
test_accuracy = np.mean(np.array(test_results['predictions']) == test_labels)
print(f"Test Accuracy: {test_accuracy:.4f}")

lstm_service.stop()

## 7. Compare All Results

In [ ]:
# Create comparison summary
summary = pd.DataFrame({
    'Metric': ['Best Tuning Score', 'CV Accuracy', 'CV F1-Score', 
               'Training Val Acc', 'Test Accuracy'],
    'Value': [
        f"{lstm_tuning_results['bestScore']:.4f}",
        f"{avg_metrics['accuracy']:.4f} ± {avg_metrics['std_accuracy']:.4f}",
        f"{avg_metrics['f1Score']:.4f} ± {avg_metrics['std_f1Score']:.4f}",
        f"{training_results['bestValAccuracy']:.4f}",
        f"{test_accuracy:.4f}"
    ]
})

print("\n" + "="*50)
print("PERFORMANCE SUMMARY")
print("="*50)
print(summary.to_string(index=False))

## 8. Save Complete Report

In [ ]:
# Create comprehensive report
complete_report = {
    'experiment': {
        'date': pd.Timestamp.now().isoformat(),
        'dataset': {
            'samples': int(n_samples),
            'features': int(n_features),
            'classes': int(n_classes)
        }
    },
    'hyperparameter_tuning': {
        'n_trials': len(tuning_data['allTrials']),
        'best_score': float(lstm_tuning_results['bestScore']),
        'best_params': best_params
    },
    'cross_validation': {
        'n_folds': 5,
        'accuracy': f"{avg_metrics['accuracy']:.4f} ± {avg_metrics['std_accuracy']:.4f}",
        'f1_score': f"{avg_metrics['f1Score']:.4f} ± {avg_metrics['std_f1Score']:.4f}",
        'precision': float(avg_metrics['precision']),
        'recall': float(avg_metrics['recall'])
    },
    'final_training': {
        'best_val_accuracy': float(training_results['bestValAccuracy']),
        'model_path': training_results['checkpointPath'],
        'epochs_trained': len(history['train_loss'])
    },
    'test_results': {
        'accuracy': float(test_accuracy),
        'test_samples': int(len(test_labels))
    }
}

# Save report
report_path = '../model_training_report.json'
with open(report_path, 'w') as f:
    json.dump(complete_report, f, indent=2)

print(f"\n✓ Complete report saved to: {report_path}")
print("\n" + json.dumps(complete_report, indent=2))

## 9. Cleanup

In [ ]:
# Stop training service
trainer.stop()
print("✓ Training service stopped")

## Summary

In this notebook, you learned:
1. ✓ Hyperparameter tuning with Optuna
2. ✓ K-Fold cross-validation
3. ✓ Model training with early stopping
4. ✓ Visualizing training progress
5. ✓ Loading and testing trained models
6. ✓ Creating comprehensive reports

**Key Takeaways:**
- Hyperparameter tuning improves model performance significantly
- Cross-validation provides robust performance estimates
- Early stopping prevents overfitting
- Training history helps understand model behavior

**Next Steps:**
- Apply to real datasets (NSL-KDD, UNSW-NB15)
- Tune CNN model
- Compare LSTM vs CNN performance
- Deploy trained models for inference